In [16]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-3-5-haiku-latest"

In [17]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [18]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")

    answer = chat(messages, stop_sequences=["```"])
    return json.loads(answer)

In [19]:
# Generate a dataset and store it on a json file
dataset = generate_dataset()

with open("dataset.json", "w") as f:
    json.dump(dataset, f, indent=2)


In [20]:
def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    prompt = f"""
    Please solve the following task:

    {test_case["task"]}
    """

    messages = []

    add_user_message(messages, prompt)

    output = chat(messages)
    return output

In [21]:
def grade_by_model(test_case, output):
    eval_prompt = f"""
    You are an expert AWS code reviewer. Your task is to evaluate the following AI-generated solution.

    Original Task:
    <task>
    {test_case["task"]}
    </task>

    Solution to Evaluate:
    <solution>
    {output}
    </solution>

    Output Format
    Provide your evaluation as a structured JSON object with the following fields, in this specific order:
    - "strengths": An array of 1-3 key strengths
    - "weaknesses": An array of 1-3 key areas for improvement
    - "reasoning": A concise explanation of your overall assessment
    - "score": A number between 1-10

    Respond with JSON. Keep your response concise and direct.
    Example response shape:
    {{
        "strengths": string[],
        "weaknesses": string[],
        "reasoning": string,
        "score": number
    }}
    """

    messages = []

    add_user_message(messages, eval_prompt)

    add_assistant_message(messages, "```json")

    eval_text = chat(messages, stop_sequences=["```"])

    return json.loads(eval_text)

In [22]:
def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)

    # TODO: Grading 
    model_grade = grade_by_model(test_case, output)
    score = model_grade["score"]
    reasoning = model_grade["reasoning"]

    return {
        "output": output,
        "test_case": test_case,
        "score": score,
        "reasoning": reasoning
    }

In [27]:
from statistics import mean

def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []

    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    average_score = mean([result["score"] for result in results])
    print(average_score)    

    return results

In [28]:
# Run eval pipeline

with open("dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

8


In [29]:
print(json.dumps(results, indent=2))

[
  {
    "output": "Here's a Python function to extract the AWS region from an EC2 instance ARN:\n\n```python\ndef extract_region_from_arn(arn):\n    \"\"\"\n    Extract the AWS region from an EC2 instance ARN.\n    \n    Args:\n        arn (str): The full ARN of an EC2 instance\n    \n    Returns:\n        str: The AWS region extracted from the ARN\n        None: If the ARN is invalid or does not contain a region\n    \"\"\"\n    # Check if the ARN is valid and starts with the expected prefix\n    if not arn or not arn.startswith('arn:aws:ec2:'):\n        return None\n    \n    # Split the ARN into parts\n    arn_parts = arn.split(':')\n    \n    # Check if the ARN has enough parts\n    if len(arn_parts) < 4:\n        return None\n    \n    # The region is the 4th part of the ARN\n    region = arn_parts[3]\n    \n    return region\n\n# Example usage\ndef main():\n    # Test cases\n    arns = [\n        'arn:aws:ec2:us-west-2:123456789012:instance/i-1234567890abcdef0',\n        'arn:a